### 导入包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### 导入数据并归一化

In [2]:
data=pd.read_csv('./clean-data.csv')
data=data.drop(['Unnamed: 0'],axis=1)
X=data.drop(['Group'],axis=1)
y=data['Group']
column=X.columns
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)
X=pd.DataFrame(data=X,columns=column)

### 划分训练集测试集

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=None)
print(f'Train shape : {X_train.shape}\nTest shape: {X_test.shape}')

Train shape : (132, 203)
Test shape: (33, 203)


In [ ]:
#只在测试集上测试
def testpredict(estimator,Xtrain,ytrain,Xtest,ytest):
    print("只在测试集上测试")
    model=estimator.fit(Xtrain,ytrain)
    y_pred_train=model.predict(Xtrain)
    y_pred_test=model.predict(Xtest)
    confusion=confusion_matrix(ytest,y_pred_test)
    acc=accuracy_score(ytest, y_pred_test)
    rec=recall_score(ytest, y_pred_test)
    pre=precision_score(ytest, y_pred_test)
    f1=f1_score(ytest, y_pred_test)
    print('the confusion_matrix of the model is : \n',confusion)
    print('the accuracy of the model on testing set is ：',acc)
    print("the precision score of the model on testing set is : ",pre )
    print("the recall score of the model on testing set is :", rec)
    print('the f1_score of the model on testing set is :',f1)
    
    return acc,pre,rec,f1

In [4]:
#在整个数据集上交叉验证
def crosspredict(estimator,Xtrain,ytrain,cv):
    print("在整个数据集上交叉验证")
    y_pred_cross=cross_val_predict(estimator,Xtrain,ytrain,cv=cv)
    confusion_cross=confusion_matrix(ytrain,y_pred_cross)
    acc=accuracy_score(ytrain,y_pred_cross)
    pre=precision_score(ytrain, y_pred_cross)
    rec=recall_score(ytrain, y_pred_cross)
    f1=f1_score(ytrain, y_pred_cross)
    print('the confusion_matrix of the model is : \n',confusion_cross)
    print('the accuracy of the model is : ',acc)
    print("the precision score of the model is : ", pre)
    print("the recall score of the model is :", rec)
    print('the f1_score of the model  is :',f1)
    return acc,pre,rec,f1

### 特征提取

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
def searchNFeatures(estimator,nEnd = 11):
    n_feature_range = range(1,nEnd+1)
    accmax1=0
    premax1=0
    recmax1=0
    f1max1=0
    featuremax1=0
    feature1=[]
    accmax2=0
    premax2=0
    recmax2=0
    f1max2=0
    featuremax2=0
    feature2=[]
    for n in n_feature_range:
        print("N features:", n)
        sel_seq = SequentialFeatureSelector(estimator=estimator, n_features_to_select=n)
        temp=sel_seq.fit(X, y)
        sel_seq_mask= X.columns[temp.get_support()]
        print('features:',sel_seq_mask)
        X_new=sel_seq.fit_transform(X,y)
        X_train1,X_test1,y_train1,y_test1 = train_test_split(X_new,y,test_size=0.2,random_state=42,stratify=None)
        acc1,pre1,rec1,f11=testpredict(estimator,X_train1,y_train1,X_test1,y_test1)
        acc2,pre2,rec2,f12=crosspredict(estimator,X_new,y,10)
        if f11>f1max1:
            f1max1=f11
            featuremax1=n
            accmax1=acc1
            premax1=pre1
            recmax1=rec1
            feature1=sel_seq_mask
        if f12>f1max2:
            f1max2=f12
            featuremax2=n
            accmax2=acc2
            premax2=pre2
            recmax2=rec2
            feature2=sel_seq_mask
    print("\n")
    print("只在测试集上测试：")
    print('feature :',feature1)
    print('the best feature number is:',featuremax1)
    print('the best accuracy is:',accmax1)
    print('the best precision is:',premax1)
    print('the best recall is:',recmax1)
    print('the best f1_score is:',f1max1)
    
    print("整个数据集上交叉验证：")
    print('feature :',feature2)
    print('the best feature number is:',featuremax2)
    print('the best accuracy is:',accmax2)
    print('the best precision is:',premax2)
    print('the best recall is:',recmax2)
    print('the best f1_score is:',f1max2)
    print("\n")

In [ ]:
model=[SVC(),RandomForestClassifier(),KNeighborsClassifier(),AdaBoostClassifier(),GaussianNB(),BernoulliNB(),
       LogisticRegression(),GradientBoostingClassifier(),XGBClassifier(),lgb.LGBMClassifier(),DecisionTreeClassifier()]


In [ ]:
for i in range(len(model)):
    print("the model is :",model[i])
    searchNFeatures(model[i],10)
    print("\n")

the model is : SVC()


只在测试集上测试：
feature : Index(['APG_0101010010M', 'APG_0101010028P', 'APG_0101010036M',
       'APG_0101010055P', 'APG_0101010179P', 'APG_0101010254P'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.7575757575757576
the best precision is: 0.9090909090909091
the best recall is: 0.5882352941176471
the best f1_score is: 0.7142857142857143
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010021M', 'APG_0101010028P',
       'APG_0101010033M', 'APG_0101010036M', 'APG_0101010055P',
       'APG_0101010085P', 'APG_0101010134M', 'APG_0101010179P',
       'APG_0101010254P'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.793939393939394
the best precision is: 0.8840579710144928
the best recall is: 0.7011494252873564
the best f1_score is: 0.782051282051282




the model is : RandomForestClassifier()


只在测试集上测试：
feature : Index(['Gender', 'APG_0101010053M', 'APG_0101010095M', 'APG_0101010133P',
       'APG_0101010192M', 'APG_0101010262P', 'APG_0101010299P',
       'APG_0101010312P', 'APG_0101010354P'],
      dtype='object')
the best feature number is: 9
the best accuracy is: 0.8484848484848485
the best precision is: 0.9285714285714286
the best recall is: 0.7647058823529411
the best f1_score is: 0.8387096774193549
整个数据集上交叉验证：
feature : Index(['Gender', 'APG_0101010014M', 'APG_0101010026P', 'APG_0101010053M',
       'APG_0101010055P', 'APG_0101010176P', 'APG_0101010262P',
       'APG_0101010288P'],
      dtype='object')
the best feature number is: 8
the best accuracy is: 0.806060606060606
the best precision is: 0.8767123287671232
the best recall is: 0.735632183908046
the best f1_score is: 0.8




the model is : KNeighborsClassifier()


只在测试集上测试：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010053M',
       'APG_0101010055P', 'APG_0101010176P', 'APG_0101010291P',
       'APG_0101010292P', 'APG_0101010356P', 'APG_0101020028P'],
      dtype='object')
the best feature number is: 9
the best accuracy is: 0.8181818181818182
the best precision is: 0.9230769230769231
the best recall is: 0.7058823529411765
the best f1_score is: 0.8000000000000002
整个数据集上交叉验证：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010012P',
       'APG_0101010053M', 'APG_0101010055P', 'APG_0101010176P',
       'APG_0101010291P', 'APG_0101010292P', 'APG_0101010356P',
       'APG_0101020028P'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.8121212121212121
the best precision is: 0.9242424242424242
the best recall is: 0.7011494252873564
the best f1_score is: 0.7973856209150327




the model is : AdaBoostClassifier()


只在测试集上测试：
feature : Index(['APG_0101010002M', 'APG_0101010032M', 'APG_0101010049P',
       'APG_0101010055P', 'APG_0101010199P', 'APG_0101010320P',
       'APG_0101010342M', 'APG_0101010364P'],
      dtype='object')
the best feature number is: 8
the best accuracy is: 0.8181818181818182
the best precision is: 0.8235294117647058
the best recall is: 0.8235294117647058
the best f1_score is: 0.8235294117647058
整个数据集上交叉验证：
feature : Index(['APG_0101010002M', 'APG_0101010032M', 'APG_0101010055P',
       'APG_0101010199P', 'APG_0101010320P', 'APG_0101010342M',
       'APG_0101010364P'],
      dtype='object')
the best feature number is: 7
the best accuracy is: 0.7696969696969697
the best precision is: 0.855072463768116
the best recall is: 0.6781609195402298
the best f1_score is: 0.7564102564102564




the model is : GaussianNB()


只在测试集上测试：
feature : Index(['APG_0101010055P', 'APG_0101010081M'], dtype='object')
the best feature number is: 2
the best accuracy is: 0.7575757575757576
the best precision is: 1.0
the best recall is: 0.5294117647058824
the best f1_score is: 0.6923076923076924
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010055P', 'APG_0101010057M',
       'APG_0101010081M', 'APG_0101010085P', 'APG_0101010199P',
       'APG_0101010264P', 'APG_0101010279P', 'APG_0101010300P'],
      dtype='object')
the best feature number is: 9
the best accuracy is: 0.8363636363636363
the best precision is: 0.9545454545454546
the best recall is: 0.7241379310344828
the best f1_score is: 0.823529411764706




the model is : BernoulliNB()


只在测试集上测试：
feature : Index(['APG_0101010054M', 'APG_0101010116P'], dtype='object')
the best feature number is: 2
the best accuracy is: 0.6060606060606061
the best precision is: 0.6111111111111112
the best recall is: 0.6470588235294118
the best f1_score is: 0.6285714285714287
整个数据集上交叉验证：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010054M',
       'APG_0101010116P', 'APG_0101010335P', 'APG_0101010346P'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.7515151515151515
the best precision is: 0.7738095238095238
the best recall is: 0.7471264367816092
the best f1_score is: 0.760233918128655




the model is : LogisticRegression()


只在测试集上测试：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010039M',
       'APG_0101010071P', 'APG_0101010145P', 'APG_0101010167M',
       'APG_0101010183P', 'APG_0101010363P'],
      dtype='object')
the best feature number is: 8
the best accuracy is: 0.7575757575757576
the best precision is: 0.8
the best recall is: 0.7058823529411765
the best f1_score is: 0.7500000000000001
整个数据集上交叉验证：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010039M',
       'APG_0101010071P', 'APG_0101010145P', 'APG_0101010167M',
       'APG_0101010183P', 'APG_0101010363P'],
      dtype='object')
the best feature number is: 8
the best accuracy is: 0.7333333333333333
the best precision is: 0.7362637362637363
the best recall is: 0.7701149425287356
the best f1_score is: 0.752808988764045




the model is : GradientBoostingClassifier()


只在测试集上测试：
feature : Index(['APG_0101010055P', 'APG_0101010176P', 'APG_0101010266P',
       'APG_0101010312P'],
      dtype='object')
the best feature number is: 4
the best accuracy is: 0.7878787878787878
the best precision is: 0.8571428571428571
the best recall is: 0.7058823529411765
the best f1_score is: 0.7741935483870968
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010026P', 'APG_0101010041M',
       'APG_0101010055P', 'APG_0101010266P', 'APG_0101010300P'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.793939393939394
the best precision is: 0.8441558441558441
the best recall is: 0.7471264367816092
the best f1_score is: 0.7926829268292682




the model is : XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
只在测试集上测试：
feature : Index(['Gender', 'APG_0101010012P', 'APG_0101010053M', 'APG_0101010262P'], dtype='object')
the best feature number is: 4
the best accuracy is: 0.7878787878787878
the best precision is: 0.7777777777777778
the best recall is: 0.8235294117647058
the best f1_score is: 0.7999999999999999
整个数据集上交叉验证：
feature : Index(['Gender', 'APG_0101010012P', 'APG_0101010053M', 'APG_0101010257P',
       'APG_0101010262P', 'APG_0101010294P', 'APG_0101010360M'],
      dtype='object')
the best feature number is: 7
the best accuracy is: 0.703030303030303
the best precision is: 0.7159090909090909
the best recall is: 0.7241379310344828
the best f1_score is: 0.72


the model is : LGBMClassifier()


只在测试集上测试：
feature : Index(['APG_0101010036M', 'APG_0101010055P', 'APG_0101010130P',
       'APG_0101010145P', 'APG_0101010254P', 'APG_0101010300P',
       'APG_0101010305P', 'APG_0101010322P', 'APG_0101010333P',
       'APG_0101010344M'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.7272727272727273
the best precision is: 0.7857142857142857
the best recall is: 0.6470588235294118
the best f1_score is: 0.7096774193548386
整个数据集上交叉验证：
feature : Index(['APG_0101010036M', 'APG_0101010055P', 'APG_0101010130P',
       'APG_0101010145P', 'APG_0101010254P', 'APG_0101010300P',
       'APG_0101010305P', 'APG_0101010322P', 'APG_0101010333P',
       'APG_0101010344M'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.7757575757575758
the best precision is: 0.8205128205128205
the best recall is: 0.735632183908046
the best f1_score is: 0.7757575757575758


the model is : DecisionTreeClassifier()

只在测试集上测试：
feature : Index(['APG_0101010006P', 'APG_0101010055P', 'APG_0101010063M',
       'APG_0101010176P', 'APG_0101010274P', 'APG_0101010289P'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.7575757575757576
the best precision is: 0.7368421052631579
the best recall is: 0.8235294117647058
the best f1_score is: 0.7777777777777778
整个数据集上交叉验证：
feature : Index(['APG_0101010055P', 'APG_0101010063M', 'APG_0101010139P',
       'APG_0101010176P', 'APG_0101010274P'],
      dtype='object')
the best feature number is: 5
the best accuracy is: 0.793939393939394
the best precision is: 0.8354430379746836
the best recall is: 0.7586206896551724
the best f1_score is: 0.7951807228915663

In [ ]:
#调参后的模型选出最优参数

In [ ]:
searchNFeatures(SVC(C=1, coef0=10, degree=4, kernel='poly', probability=True),10)

只在测试集上测试：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010055P',
       'APG_0101010247P', 'APG_0101010300P', 'APG_0101010332P'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.7878787878787878
the best precision is: 0.9166666666666666
the best recall is: 0.6470588235294118
the best f1_score is: 0.7586206896551724
整个数据集上交叉验证：
feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010037P',
       'APG_0101010039M', 'APG_0101010055P', 'APG_0101010143P',
       'APG_0101010247P', 'APG_0101010300P', 'APG_0101010332P'],
      dtype='object')
the best feature number is: 9
the best accuracy is: 0.8
the best precision is: 0.875
the best recall is: 0.7241379310344828
the best f1_score is: 0.7924528301886793

In [ ]:
searchNFeatures(RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,random_state=90),10)

只在测试集上测试：
feature : Index(['APG_0101010048P', 'APG_0101010055P', 'APG_0101010067M',
       'APG_0101010171M', 'APG_0101010262P', 'APG_0101010266P',
       'APG_0101010344M'],
      dtype='object')
the best feature number is: 7
the best accuracy is: 0.8181818181818182
the best precision is: 1.0
the best recall is: 0.6470588235294118
the best f1_score is: 0.7857142857142858
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010048P', 'APG_0101010055P',
       'APG_0101010067M', 'APG_0101010092P', 'APG_0101010098M',
       'APG_0101010171M', 'APG_0101010262P', 'APG_0101010266P',
       'APG_0101010344M'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.7818181818181819
the best precision is: 0.8227848101265823
the best recall is: 0.7471264367816092
the best f1_score is: 0.7831325301204819

In [ ]:
searchNFeatures(KNeighborsClassifier(n_neighbors=13),10)

只在测试集上测试：
feature : Index(['APG_0101010035M', 'APG_0101010055P', 'APG_0101010159P',
       'APG_0101010176P', 'APG_0101010273P', 'APG_0101010300P'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.8181818181818182
the best precision is: 0.9230769230769231
the best recall is: 0.7058823529411765
the best f1_score is: 0.8000000000000002
整个数据集上交叉验证：
feature : Index(['APG_0101010032M', 'APG_0101010035M', 'APG_0101010039M',
       'APG_0101010055P', 'APG_0101010097M', 'APG_0101010159P',
       'APG_0101010173P', 'APG_0101010176P', 'APG_0101010273P',
       'APG_0101010300P'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.793939393939394
the best precision is: 0.9076923076923077
the best recall is: 0.6781609195402298
the best f1_score is: 0.7763157894736842

In [ ]:
searchNFeatures(AdaBoostClassifier(learning_rate=0.1),10)

只在测试集上测试：
feature : Index(['APG_0101010033M', 'APG_0101010063M', 'APG_0101010147M',
       'APG_0101010192M'],
      dtype='object')
the best feature number is: 4
the best accuracy is: 0.7878787878787878
the best precision is: 0.7777777777777778
the best recall is: 0.8235294117647058
the best f1_score is: 0.7999999999999999
整个数据集上交叉验证：
feature : Index(['APG_0101010033M', 'APG_0101010063M', 'APG_0101010067M',
       'APG_0101010147M', 'APG_0101010192M', 'APG_0101010236M'],
      dtype='object')
the best feature number is: 6
the best accuracy is: 0.7757575757575758
the best precision is: 0.7659574468085106
the best recall is: 0.8275862068965517
the best f1_score is: 0.7955801104972374


In [ ]:
searchNFeatures(LogisticRegression(C=0.1),10)

只在测试集上测试：
feature : Index(['APG_0101010010M', 'APG_0101010030M', 'APG_0101010041M',
       'APG_0101010043P', 'APG_0101010137P', 'APG_0101010150P',
       'APG_0101010152M', 'APG_0101010168P', 'APG_0101010176P',
       'APG_0101010356P'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.7272727272727273
the best precision is: 0.75
the best recall is: 0.7058823529411765
the best f1_score is: 0.7272727272727272
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010030M', 'APG_0101010041M',
       'APG_0101010043P', 'APG_0101010137P', 'APG_0101010150P',
       'APG_0101010152M', 'APG_0101010168P', 'APG_0101010176P',
       'APG_0101010356P'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.7575757575757576
the best precision is: 0.7831325301204819
the best recall is: 0.7471264367816092
the best f1_score is: 0.7647058823529411

In [ ]:
searchNFeatures(XGBClassifier(booster='gblinear',learning_rate=0.3,n_estimators=10),10)

只在测试集上测试：
feature : Index(['APG_0101010002M', 'APG_0101010043P', 'APG_0101010052M',
       'APG_0101010055P', 'APG_0101010235P', 'APG_0101010262P',
       'APG_0101010266P', 'APG_0101010273P', 'APG_0101010338M',
       'APG_0101010354P'],
      dtype='object')
the best feature number is: 10
the best accuracy is: 0.8181818181818182
the best precision is: 0.8666666666666667
the best recall is: 0.7647058823529411
the best f1_score is: 0.8125
整个数据集上交叉验证：
feature : Index(['APG_0101010002M', 'APG_0101010055P', 'APG_0101010266P',
       'APG_0101010292P'],
      dtype='object')
the best feature number is: 4
the best accuracy is: 0.7575757575757576
the best precision is: 0.8133333333333334
the best recall is: 0.7011494252873564
the best f1_score is: 0.7530864197530864

In [ ]:
searchNFeatures(XGBClassifier(booster='gblinear',learning_rate=0.3,max_depth=20,n_estimators=10),10)

只在测试集上测试：
feature : Index(['APG_0101010078P'], dtype='object')
the best feature number is: 1
the best accuracy is: 0.7272727272727273
the best precision is: 0.75
the best recall is: 0.7058823529411765
the best f1_score is: 0.7272727272727272
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010037P', 'APG_0101010063M',
       'APG_0101010078P', 'APG_0101010150P', 'APG_0101010230P',
       'APG_0101010279P', 'APG_0101010312P', 'APG_0101020013M'],
      dtype='object')
the best feature number is: 9
the best accuracy is: 0.7454545454545455
the best precision is: 0.8260869565217391
the best recall is: 0.6551724137931034
the best f1_score is: 0.7307692307692308

In [ ]:
searchNFeatures(lgb.LGBMClassifier(max_depth=3, n_estimators=300),10)

只在测试集上测试：
feature : Index(['APG_0101010078P', 'APG_0101010192M', 'APG_0101010226P'], dtype='object')
the best feature number is: 3
the best accuracy is: 0.696969696969697
the best precision is: 0.6666666666666666
the best recall is: 0.8235294117647058
the best f1_score is: 0.7368421052631577
整个数据集上交叉验证：
feature : Index(['APG_0101010036M', 'APG_0101010078P', 'APG_0101010130P',
       'APG_0101010192M', 'APG_0101010226P', 'APG_0101010242P',
       'APG_0101010265P'],
      dtype='object')
the best feature number is: 7
the best accuracy is: 0.696969696969697
the best precision is: 0.7078651685393258
the best recall is: 0.7241379310344828
the best f1_score is: 0.7159090909090908

In [ ]:
searchNFeatures(DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6),10)

只在测试集上测试：
feature : Index(['Age', 'Gender', 'APG_0101010010M', 'APG_0101010055P',
       'APG_0101010063M', 'APG_0101010275P', 'APG_0101010341M',
       'APG_0101010343P', 'APG_0101010354P'],
      dtype='object')
the best feature number is: 9
the best accuracy is: 0.7878787878787878
the best precision is: 0.8125
the best recall is: 0.7647058823529411
the best f1_score is: 0.787878787878788
整个数据集上交叉验证：
feature : Index(['APG_0101010010M', 'APG_0101010055P', 'APG_0101010063M',
       'APG_0101010341M', 'APG_0101010354P'],
      dtype='object')
the best feature number is: 5
the best accuracy is: 0.7878787878787878
the best precision is: 0.8611111111111112
the best recall is: 0.7126436781609196
the best f1_score is: 0.779874213836478